In [169]:
import pandas as pd
import numpy as np
import os
from os.path import exists
import opendatasets as od
import shutil
import wbgapi as wb

import pymc as pm
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import arviz as az

import requests
from io import BytesIO

config = {
    'figure.figsize':(14,4),
    'figure.constrained_layout.use':True,
    'figure.facecolor':'w',
    'axes.grid':True,
    'grid.linewidth':0.5,
    'grid.linestyle':'--',
    'axes.spines.top':False,
    'axes.spines.bottom':False,
    'axes.spines.left':False,
    'axes.spines.right':False
}

plt.rcParams.update(config)

SEED = sum(list(map(ord, 'olympics_bayes')))
rng = np.random.default_rng(SEED)

In [51]:
import pycountry

def get_country_code(country_name):
    # First, try to find the country in the current countries list
    try:
        country = pycountry.countries.lookup(country_name)
        return country.alpha_2
    except LookupError:
        # If not found, try to find it in the historic countries list
        try:
            historic_country = pycountry.historic_countries.lookup(country_name)
            return historic_country.alpha_2
        except LookupError:
            return "Error"

In [241]:
from sklearn.linear_model import LinearRegression

def fill_future_values_last_five_years(group, value ='YR'):
    # Sort by time to ensure correct order
    group = group.sort_index(level=1)
    
    # Identify the last time period with a non-NaN value
    last_valid_index = group[value].last_valid_index()
    
    if last_valid_index:
        last_valid_time = last_valid_index[1]
        
        # Consider the last ten years of available non-NaN data for model fitting
        valid_data = group.loc[group.index.get_level_values(1) <= last_valid_time].dropna(subset=[value])
        
        # Filter to keep only the last ten years
        last_ten_years = valid_data.tail(5)
        
        if len(last_ten_years) >= 2:  # At least two data points are needed to fit a linear model
            X = last_ten_years.index.get_level_values(1).values.reshape(-1, 1)  # Time period
            y = last_ten_years[value].values  # Observed values
            model = LinearRegression().fit(X, y)
            
            # Identify future periods to predict (after the last valid time)
            future_data = group.loc[group.index.get_level_values(1) > last_valid_time]
            if not future_data.empty:
                future_periods = future_data.index.get_level_values(1).values.reshape(-1, 1)
                
                # Predict the next value as an extension from the last known value
                future_predictions = model.predict(future_periods)
                
                # Set the predicted value for future periods
                group.loc[future_data.index, value] = future_predictions
                
    return group

def fill_future_values_last_ten_years(group, value ='YR'):
    # Sort by time to ensure correct order
    group = group.sort_index(level=1)
    
    # Identify the last time period with a non-NaN value
    last_valid_index = group[value].last_valid_index()
    
    if last_valid_index:
        last_valid_time = last_valid_index[1]
        
        # Consider the last ten years of available non-NaN data for model fitting
        valid_data = group.loc[group.index.get_level_values(1) <= last_valid_time].dropna(subset=[value])
        
        # Filter to keep only the last ten years
        last_ten_years = valid_data.tail(10)
        
        if len(last_ten_years) >= 2:  # At least two data points are needed to fit a linear model
            X = last_ten_years.index.get_level_values(1).values.reshape(-1, 1)  # Time period
            y = last_ten_years[value].values  # Observed values
            model = LinearRegression().fit(X, y)
            
            # Identify future periods to predict (after the last valid time)
            future_data = group.loc[group.index.get_level_values(1) > last_valid_time]
            if not future_data.empty:
                future_periods = future_data.index.get_level_values(1).values.reshape(-1, 1)
                
                # Predict the next value as an extension from the last known value
                future_predictions = model.predict(future_periods)
                
                # Set the predicted value for future periods
                group.loc[future_data.index, value] = future_predictions
                
    return group

# Olympic Data

## Host Country

In [52]:
df = pd.read_csv("../Data/raw/olympic_hosts.csv")

df['game_start_date'] = pd.to_datetime(df['game_start_date'])

# Year is different than game year because the Tokyo 2020 games were held in 2021
df['year'] = df['game_start_date'].dt.year

# Equestrian events took place in Sweden however the games overall were in Australia
df.replace({'game_location': "Australia, Sweden"}, "Australia", inplace=True)

df['host_city'] = df['game_name'].str[:-5]

df.rename(columns={"game_slug":"game_id","game_location":"host_country"}, inplace=True)

df['host_country'] = df['host_country'].replace({"Federal Republic of Germany":"Germany","Great Britain":"United Kingdom",
"Republic of Korea":"Korea, Republic of", "Yugoslavia":"Yugoslavia, (Socialist) Federal Republic of", "USSR":"USSR, Union of Soviet Socialist Republics"})
df["host_country"] = df["host_country"].map(lambda x: get_country_code(x))

game_vars = ["game_id", "year","game_name","host_country","host_city","game_season"]

df_host_all = df[game_vars]
df_host_summer = df.query("game_season == 'Summer'")[game_vars]
df_host_winter = df.query("game_season == 'Winter'")[game_vars]

df_host_all.to_csv("../Data/clean/hosts_all")
df_host_summer.to_csv("../Data/clean/hosts_summer")
df_host_winter.to_csv("../Data/clean/hosts_winter")

## Participants & Medal Count

In [53]:
df = pd.read_csv("../Data/raw/olympic_results.csv")

df['country_name'] = df['country_name'].replace(
{"Brunei":"Brunei Darussalam",
"Cape Verde":"Cabo Verde",
"Chinese Taipei":"Taiwan",
"Czechoslovakia":"Serbia and Montenegro", #This is not correct mapping (obvs), however maps to CS which was re-used
"Democratic Republic of the Congo":"Congo, The Democratic Republic of the",
"German Democratic Republic (Germany)":"German Democratic Republic",
"Great Britain":"United Kingdom",
"Hong Kong, China":"Hong Kong",
"Ivory Coast":"Côte d'Ivoire",
'Olympic Athletes from Russia':"Russian Federation",
"Palestine":"Palestine, State of",
"ROC":"Russian Federation",
"Republic of Korea":"South Korea",
"Rhodesia":"Southern Rhodesia",
"Saar":"Germany",
"South Vietnam":"Viet Nam",
"Soviet Union":"USSR, Union of Soviet Socialist Republics",
"Swaziland":"Eswatini",
"The Former Yugoslav Republic of Macedonia":"North Macedonia",
"Turkey":"Türkiye",
"US Virgin Islands":'Virgin Islands, U.S.',
"Unified Team":"USSR, Union of Soviet Socialist Republics",
"Virgin Islands, US":'Virgin Islands, U.S.',
"Yemen Arab Republic":"Republic of Yemen",
"Yemen Democratic Republic":"Yemen, Democratic, People's Democratic Republic of",
"Yugoslavia":"Yugoslavia, (Socialist) Federal Republic of"})

#Early, Defunct, and Otherwise Minor
a = ["Australasia", #Joint Aussie-Kiwi team
"Bohemia", #The constituent kingdoms of the Austro-Hungarian Empire competed seperately
"Korea Team", #DPRK and ROK had a joint team for an event in 2018
"Malaya", #Formed Malaysia
"North Borneo", #Formed Malaysia
"Newfoundland", #Prior to joining Canada
"United Arab Republic" #Brief merger of Syria and Egypt
"MIX", #Earliest Olympics allowed teams from multiple countries
"West Indies Federation" #Brief federation of former British colonies in the Caribbean
]

df.loc[(df['slug_game'] == "barcelona-1992") & (df['country_name'] == "Independent Olympic Athletes"), 'country_name'] = "USSR, Union of Soviet Socialist Republics"
df.loc[(df['slug_game'] == "sydney-2000") & (df['country_name'] == "Independent Olympic Athletes"), 'country_name'] = "Timor-Leste"
df.loc[(df['slug_game'] == "london-2012") & (df['country_name'] == "Independent Olympic Athletes"), 'country_name'] = "Netherlands Antilles"
df.loc[(df['slug_game'] == "sochi-2014") & (df['country_name'] == "Independent Olympic Athletes"), 'country_name'] = "India"
df.loc[(df['slug_game'] == "rio-2016") & (df['country_name'] == "Independent Olympic Athletes"), 'country_name'] = "Kuwait"

df["country_code"] = df["country_name"].map(lambda x: get_country_code(x))

df.loc[df['country_name'] == "Refugee Olympic Athletes", 'country_code'] = "UN"
df.loc[df['country_name'] == "Refugee Olympic Team", 'country_code'] = "UN"
df.loc[df['country_name'] == "Kosovo", 'country_code'] = "XK"

df['Gold'] = 0
df.loc[df['medal_type'] == "GOLD", 'Gold'] = 1

df['Silver'] = 0
df.loc[df['medal_type'] == "SILVER", 'Silver'] = 1

df['Bronze'] = 0
df.loc[df['medal_type'] == "BRONZE", 'Bronze'] = 1

df_medal_count = df.groupby(["slug_game","country_code"]).sum()[['Gold','Silver','Bronze']]
df_medal_count.to_csv("../Data/clean/medal_count")

# Independent Vars

## GDP per Capita

In [323]:
df = pd.read_csv('../Data/raw/NY_GDP_PCAP_CD.csv')

for _ in range(10):
    df[df.columns[-1][:-4] + str(int(df.columns[-1][-4:])+1)] = np.NaN

df["country_code"] = df["economy"].map(lambda x: get_country_code(x))

df = df.loc[df['country_code'] != "Error"].iloc[:,1:]
df = pd.wide_to_long(df, ["YR"], i="country_code", j="year")

df = np.log(df).groupby(level=0).apply(fill_future_values_last_five_years)

# Resetting the index to avoid duplicate levels
df = df.reset_index(level=0, drop=True)

df = df.reset_index().pivot(index="year",columns="country_code",values="YR")
df = df.interpolate()

df_gdp = np.e**df

df_gdp.to_csv("../Data/clean/gdp")

## Population

In [330]:
df = pd.read_csv('../Data/raw/SP_POP_TOTL.csv')

for _ in range(10):
    df[df.columns[-1][:-4] + str(int(df.columns[-1][-4:])+1)] = np.NaN

df["country_code"] = df["economy"].map(lambda x: get_country_code(x))

df = df.loc[df['country_code'] != "Error"].iloc[:,1:]
df = pd.wide_to_long(df, ["YR"], i="country_code", j="year")

df = np.log(df).groupby(level=0).apply(fill_future_values_last_five_years)

# Resetting the index to avoid duplicate levels
df = df.reset_index(level=0, drop=True)

df = df.reset_index().pivot(index="year",columns="country_code",values="YR")
df = df.interpolate()

df = np.e**df

df_pop_total = df
df_pop = df.div(df.sum(axis=1), axis=0) * 100
df_pop.to_csv("../Data/clean/pct_pop")

## GDP Total

In [335]:
df = df_gdp*df_pop_total

df_gdp_total = df.div(df.sum(axis=1), axis=0) * 100
df_gdp_total.to_csv("../Data/clean/pct_gdp")

## Freedom Index

In [618]:
xl = pd.ExcelFile("../Data/raw/FIW_1973-2024.xls")

df = xl.parse('Country Ratings, Statuses ', skiprows=1)
df = df.rename(columns={"Year(s) Under Review":"Country"})
df = df.iloc[1:,:]

df['Country'] = df['Country'].replace(
{"Brunei":"Brunei Darussalam",
"Congo (Brazzaville)":"Congo",
"Congo (Kinshasa)":"Congo, The Democratic Republic of the",
"Cote d'Ivoire":"Côte d'Ivoire",
"Czechoslovakia":"Serbia and Montenegro", #This is not correct mapping (obvs), however maps to CS which was re-used
"Germany, E. ":"German Democratic Republic",
"Germany, W. ":"Germany",
"Micronesia":"Micronesia, Federated States of",
"Russia":"Russian Federation",
"St. Kitts and Nevis":"Saint Kitts and Nevis",
"St. Lucia":"Saint Lucia",
"St. Vincent and the Grenadines":"Saint Vincent and the Grenadines",
"The Gambia":"Gambia",
"Turkey":"Türkiye",
"USSR":"USSR, Union of Soviet Socialist Republics",
"Vietnam, N.":"Viet-Nam, Democratic Republic of",
"Vietnam, S.":"Viet Nam",
"Yemen, N.":"Republic of Yemen",
"Yemen, S.":"Yemen, Democratic, People's Democratic Republic of",
"Yugoslavia":"Yugoslavia, (Socialist) Federal Republic of"})

df["country_code"] = df["Country"].map(lambda x: get_country_code(x))

df.loc[df['Country'] == "Kosovo", 'country_code'] = "XK"

#South Africa for has odd data, I'm using the values from the next two years
df.loc[(df['country_code'] == "ZA"), 1972] = 4
df.loc[(df['country_code'] == "ZA"), "Unnamed: 2"] = 5

df = df.loc[df['country_code'] != "Error"].iloc[:,1:]

df.replace("-", np.nan, inplace=True)

df = df.set_index("country_code")

# Get list of all column sets
column_sets = [df.columns[i:i+3] for i in range(0, len(df.columns), 3)]

# Process each set of three columns
dfs = []
for columns in column_sets:
    # Extract columns for this set
    subset_df = df.loc[:, columns]  # Using .loc to work on a view of the original dataframe
    
    # Calculate sum of first column and drop the second and third columns
    new_column_name = columns[0]
    subset_df[new_column_name] = (subset_df[columns[0]] + subset_df[columns[1]])/2
    
    # Drop the second and third columns
    subset_df.drop(columns=columns[1:], inplace=True)
    
    dfs.append(subset_df)

# Concatenate all modified dataframes into a final dataframe
df = pd.concat(dfs, axis=1)

for i in df.columns:
    df.rename(columns={i:("YR"+str(i)[-4:])}, inplace=True)

df = df.groupby('country_code').mean()

df = pd.wide_to_long(df.reset_index(), ["YR"], i="country_code", j="year")

df = df.reset_index().pivot(index="year",columns="country_code",values="YR")
df = df.interpolate()

df_free = df
df_pop = df.div(df.sum(axis=1), axis=0) * 100
df_free.to_csv("../Data/clean/freedom")

/var/folders/w8/rmt0d09s7yd6wt42l__22p3h0000gn/T/ipykernel_47781/4013924470.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("-", np.nan, inplace=True)


country_code,AD,AE,AF,AG,AL,AM,AO,AR,AT,AU,...,VN,VU,WS,XK,YD,YE,YU,ZA,ZM,ZW
year,,,,,,,,,,,,,,,,,,,,,
1972,3.50000,6.0,4.5,NaN,7.0,NaN,NaN,4.5,1.0,1.0,...,4.5,NaN,3.0,NaN,7.0,4.0,6.0,4.5,5.0,5.5
1973,4.00000,5.5,6.5,NaN,7.0,NaN,NaN,2.0,1.0,1.0,...,4.5,NaN,3.0,NaN,7.0,4.5,6.0,4.5,5.0,5.5
1974,4.00000,5.5,6.5,NaN,7.0,NaN,NaN,3.0,1.0,1.0,...,4.5,NaN,3.0,NaN,7.0,4.5,6.0,4.5,4.5,5.5
1975,4.00000,5.5,6.5,NaN,7.0,NaN,6.0,3.0,1.0,1.0,...,7.0,NaN,3.0,NaN,7.0,5.5,6.0,4.5,5.0,5.5
1976,4.00000,5.0,6.5,NaN,7.0,NaN,6.0,5.5,1.0,1.0,...,7.0,NaN,3.0,NaN,7.0,5.5,6.0,4.5,5.0,5.5
1977,3.84375,5.0,6.0,NaN,7.0,NaN,7.0,6.0,1.0,1.0,...,7.0,NaN,3.0,NaN,7.0,5.5,5.5,5.5,5.0,5.5
1978,3.68750,5.0,7.0,NaN,7.0,NaN,7.0,5.5,1.0,1.0,...,7.0,NaN,3.0,NaN,6.5,5.5,5.5,5.5,5.0,5.0
1979,3.53125,5.0,7.0,NaN,7.0,NaN,7.0,5.5,1.0,1.0,...,7.0,NaN,3.0,NaN,6.5,5.5,5.5,5.5,5.0,4.0
1980,3.37500,5.0,7.0,NaN,7.0,NaN,7.0,5.5,1.0,1.0,...,7.0,2.5,3.5,NaN,6.5,5.5,5.5,5.5,5.5,3.5
